In [1]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk as nlp
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import warnings

C:\Users\acer\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
print(os.getcwd())

C:\Users\acer\AI Coursework


# Load Dataset

In [3]:
# Load fake news datasets
filePath = os.path.join(os.getcwd(),'Fake.csv')
df_fake = pd.read_csv(filePath)

In [4]:
df_fake

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [5]:
# Load true news datasets
truefilePath = os.path.join(os.getcwd(),'True.csv')
df_true = pd.read_csv(truefilePath)

In [6]:
df_true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


# Labeling News Data

In [7]:
# Add a 'class' column to label fake (0) and true (1) news
df_fake["class"] = 0
df_true["class"] = 1

In [8]:
# Display the shapes of the modified datasets
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

# Removing rows for manual testing

In [9]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis = 0, inplace = True)
    
    
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [10]:
# Display the shapes of the modified datasets
df_fake.shape, df_true.shape

((23471, 5), (21407, 5))

# Manual Testing Labeling

In [11]:
df_fake_manual_testing = df_fake_manual_testing.copy()
df_true_manual_testing = df_true_manual_testing.copy()

# Assign labels to the manual testing datasets
df_fake_manual_testing.loc[:, "class"] = 0
df_true_manual_testing.loc[:, "class"] = 1

In [12]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [13]:
df_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


# Concatenate and save manual testing datasets

In [14]:
# Concatenate manual testing datasets
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
# Save manual testing dataset to CSV
df_manual_testing.to_csv("manual_testing.csv")

In [15]:
# Concatenate the modified fake and true news datasets
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


# Merge Columns

In [16]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

# Remove Unnecessary Columns

In [17]:
# Drop unnecessary columns
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [18]:
df.isnull().sum()

text     0
class    0
dtype: int64

# Shuffle dataset

In [19]:
# Shuffle the dataset
df = df.sample(frac = 1)
df.head()

,text,class
5677,WASHINGTON (Reuters) - President Donald Trump ...,1
9470,"WASHINGTON (Reuters) - Donald Trump, the presu...",1
14113,Trayvon s dad sounds more logical and Preside...,0
4595,Rachel Maddow has a way of cutting to the poin...,0
7454,"Michigan s embattled conservative governor, Ri...",0


In [20]:
# Reset index
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)
df.head()

,text,class
0,WASHINGTON (Reuters) - President Donald Trump ...,1
1,"WASHINGTON (Reuters) - Donald Trump, the presu...",1
2,Trayvon s dad sounds more logical and Preside...,0
3,Rachel Maddow has a way of cutting to the poin...,0
4,"Michigan s embattled conservative governor, Ri...",0


# Text preprocessing

In [21]:
def wordopt(text):
    # Convert the text to lowercase
    text = text.lower()

    # Remove square brackets and content within them
    text = re.sub('\[.*?\]', '', text)
    
    # Replace non-word characters with space
    text = re.sub("\\W", " ", text) 
    
    # Remove URLs (links starting with http(s) or www)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub('<.*?>+', '', text)
    
    # Remove punctuation using string.punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    # Remove newline characters
    text = re.sub('\n', '', text)
    
    # Remove words containing digits or a combination of letters and digits
    text = re.sub('\w*\d\w*', '', text)
    return text

In [22]:
df["text"] = df["text"].apply(wordopt)

In [23]:
df.head(100)

,text,class
0,washington reuters president donald trump ...,1
1,washington reuters donald trump the presu...,1
2,trayvon s dad sounds more logical and preside...,0
3,rachel maddow has a way of cutting to the poin...,0
4,michigan s embattled conservative governor ri...,0
...,...,...
95,jerusalem reuters u s president donald tr...,1
96,did donald trump convince megyn kelly s loyal ...,0
97,the windy city is under fire for turning publi...,0
98,landowners in nebraska are understandably upse...,0


# Splitting dataset for training and testing 

In [24]:
# Split the dataset into features (x) and labels (y)
x = df["text"]
y = df["class"]
x.shape

(44878,)

In [25]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
x_train.shape

(33658,)

# Text Vectorization using TfidfVectorizer

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TfidfVectorizer object
vectorization = TfidfVectorizer()

# Build an analyzer function using the vectorizer
analyze = vectorization.build_analyzer()

# Print the analysis of the first training sample (uncomment to see the result)
# print(analyze(x_train[0]))

# Transform the training data using TfidfVectorizer
xv_train = vectorization.fit_transform(x_train)

# Transform the testing data using the same vectorizer
xv_test = vectorization.transform(x_test)

# Print the shape of the transformed training and testing datasets
print("Shape of the transformed training dataset:", xv_train.shape)
print("Shape of the transformed testing dataset:", xv_test.shape)

Shape of the transformed training dataset: (33658, 94952)
Shape of the transformed testing dataset: (11220, 94952)


# Logistic Regression Model Training and Evaluation

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Create a LogisticRegression model
lr_classifier = LogisticRegression()

# Train the LogisticRegression model
lr_classifier.fit(xv_train, y_train)

# Calculate and print the accuracy in percentage format
accuracy = lr_classifier.score(xv_test, y_test) * 100
print("The Accuracy of the Logistic Regression Model is {:.2f}%".format(accuracy))

# Print the classification report
print(classification_report(y_test, lr_classifier.predict(xv_test)))

The Accuracy of the Logistic Regression Model is 98.59%
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5884
           1       0.98      0.99      0.99      5336

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



# Decision Tree ClassifieTraining and Evaluation

In [28]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(xv_train,y_train)
print("The Accuracy of the Decision Tree Classifier Model is {}".format(dtc.score(xv_test,y_test)))
print(classification_report(y_test,dtc.predict(xv_test)))

The Accuracy of the Decision Tree Classifier Model is 0.9955436720142602
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5884
           1       1.00      0.99      1.00      5336

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



# Random Forest  Classifier Training and Evaluation

In [29]:
from sklearn.ensemble import RandomForestClassifier
rclf = RandomForestClassifier()
rclf.fit(xv_train,y_train)
print("The Accuracy of the Random Forest Classifier Model is {}".format(rclf.score(xv_test,y_test)))
print(classification_report(y_test,rclf.predict(xv_test)))

The Accuracy of the Random Forest Classifier Model is 0.9913547237076649
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5884
           1       0.99      0.99      0.99      5336

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



# Manual Testing

In [30]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_lr = lr_classifier.predict(new_xv_test)
    pred_dtc = dtc.predict(new_xv_test)
    pred_rclf = rclf.predict(new_xv_test)

return print("\n\nLR Prediction: {} \nDTC Prediction: {}  \nRFC Prediction: {}".format(output_lable(pred_lr[0]), output_lable(pred_dtc[0]), output_lable(pred_rclf[0])))

In [31]:
news = str(input())
manual_testing(news)

 BRUSSELS (Reuters) - NATO allies on Tuesday welcomed President Donald Trump s decision to commit more forces to Afghanistan, as part of a new U.S. strategy he said would require more troops and funding from America s partners. Having run for the White House last year on a pledge to withdraw swiftly from Afghanistan, Trump reversed course on Monday and promised a stepped-up military campaign against  Taliban insurgents, saying:  Our troops will fight to win .  U.S. officials said he had signed off on plans to send about 4,000 more U.S. troops to add to the roughly 8,400 now deployed in Afghanistan. But his speech did not define benchmarks for successfully ending the war that began with the U.S.-led invasion of Afghanistan in 2001, and which he acknowledged had required an   extraordinary sacrifice of blood and treasure .  We will ask our NATO allies and global partners to support our new strategy, with additional troops and funding increases in line with our own. We are confident they 



LR Prediction: Not A Fake News 
DTC Prediction: Not A Fake News  
RFC Prediction: Not A Fake News


In [32]:
news = str(input())
manual_testing(news)

 Vic Bishop Waking TimesOur reality is carefully constructed by powerful corporate, political and special interest sources in order to covertly sway public opinion. Blatant lies are often televised regarding terrorism, food, war, health, etc. They are fashioned to sway public opinion and condition viewers to accept what have become destructive societal norms.The practice of manipulating and controlling public opinion with distorted media messages has become so common that there is a whole industry formed around this. The entire role of this brainwashing industry is to figure out how to spin information to journalists, similar to the lobbying of government. It is never really clear just how much truth the journalists receive because the news industry has become complacent. The messages that it presents are shaped by corporate powers who often spend millions on advertising with the six conglomerates that own 90% of the media:General Electric (GE), News-Corp, Disney, Viacom, Time Warner, 



LR Prediction: Fake News 
DTC Prediction: Fake News  
RFC Prediction: Fake News


# Hyperparameter tuning of logistic regression

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Create empty lists to store metrics
precisions_lr = []
recalls_lr = []
f1_scores_lr = []

# Convert text data to TF-IDF features
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf = tfidf_vectorizer.transform(x_test)

#hyperparameter grid
param_grid_lr = {
    'penalty': ['l2'],
    'C': [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 20, 50]
}

# Creating the GridSearchCV object with scoring
grid_search_lr = GridSearchCV(estimator=LogisticRegression(solver='liblinear'), param_grid=param_grid_lr, scoring='accuracy', cv=5, verbose=True)
grid_search_lr.fit(x_train_tfidf, y_train)

# Print the results for all C values
results_lr = pd.DataFrame(grid_search_lr.cv_results_)

# Print the metrics for each C value and store the best hyperparameters
best_params_lr = None
best_accuracy_lr = float('-inf')

for i in range(len(results_lr)):
    params = results_lr.loc[i, 'params']
    accuracy = results_lr.loc[i, 'mean_test_score']
    
    # Train the Logistic Regression model with the current hyperparameters
    lr_model = LogisticRegression(solver='liblinear', **params)
    lr_model.fit(x_train_tfidf, y_train)

    # Predict on the test set
    y_pred_lr = lr_model.predict(x_test_tfidf)

    # Calculate precision, recall, and f1-score
    precision_lr = precision_score(y_test, y_pred_lr, average='weighted')
    recall_lr = recall_score(y_test, y_pred_lr, average='weighted')
    f1_lr = f1_score(y_test, y_pred_lr, average='weighted')

    # Print the metrics for the current C value
    print(f"\nMetrics for C = {params['C']}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision_lr}")
    print(f"Recall: {recall_lr}")
    print(f"F1-score: {f1_lr}")

    # Checking for the best result 
    if accuracy > best_accuracy_lr:
        best_accuracy_lr = accuracy
        best_params_lr = params

# Print the best hyperparameters and corresponding metrics
print("\nBest Hyperparameters (Logistic Regression):", best_params_lr)
print("Best Accuracy:", best_accuracy_lr)

Fitting 5 folds for each of 10 candidates, totalling 50 fits

Metrics for C = 0.001:
Accuracy: 0.8997859647273685
Precision: 0.9202102167000179
Recall: 0.9150623885918003
F1-score: 0.9145372760009318

Metrics for C = 0.01:
Accuracy: 0.9470554646767428
Precision: 0.9490197724250408
Recall: 0.9489304812834225
F1-score: 0.9489432410313335

Metrics for C = 0.1:
Accuracy: 0.9709131612559126
Precision: 0.971987878553203
Recall: 0.9719251336898396
F1-score: 0.9719313871562062

Metrics for C = 0.5:
Accuracy: 0.9809256185828149
Precision: 0.9822054622981666
Recall: 0.982174688057041
F1-score: 0.9821775712009553

Metrics for C = 1:
Accuracy: 0.9842235352202406
Precision: 0.9850514484451105
Recall: 0.9850267379679144
F1-score: 0.9850289348587122

Metrics for C = 2:
Accuracy: 0.9869272306555466
Precision: 0.9872640784085308
Recall: 0.9872549019607844
F1-score: 0.9872560248749355

Metrics for C = 5:
Accuracy: 0.9898983013562616
Precision: 0.9898475834732685
Recall: 0.9898395721925134
F1-score: 0.98